In [72]:
files = []
files.append(open('reuters21578/all-exchanges-strings.lc.txt','r'))
files.append(open('reuters21578/all-orgs-strings.lc.txt','r'))
files.append(open('reuters21578/all-people-strings.lc.txt','r'))
files.append(open('reuters21578/all-places-strings.lc.txt','r'))
files.append(open('reuters21578/all-topics-strings.lc.txt','r'))

In [73]:
for f in files:
    x=f.read().split('\n')
    print len(x)
    print x
    print "\n"
    

40
['amex', 'ase', 'asx', 'biffex', 'bse', 'cboe', 'cbt', 'cme', 'comex', 'cse', 'fox', 'fse', 'hkse', 'ipe', 'jse', 'klce', 'klse', 'liffe', 'lme', 'lse', 'mase', 'mise', 'mnse', 'mose', 'nasdaq', 'nyce', 'nycsce', 'nymex', 'nyse', 'ose', 'pse', 'set', 'simex', 'sse', 'stse', 'tose', 'tse', 'wce', 'zse', '']


57
['adb-africa', 'adb-asia', 'aibd', 'aid', 'anrpc', 'asean', 'atpc', 'bis', 'cipec', 'comecon', 'ec', 'eca', 'ecafe', 'ece', 'ecla', 'ecsc', 'ecwa', 'efta', 'eib', 'emcf', 'escap', 'euratom', 'fao', 'gatt', 'gcc', 'geplacea', 'iaea', 'iata', 'icco', 'ico-coffee', 'ico-islam', 'ida', 'iea', 'iisi', 'ilo', 'ilzsg', 'imco', 'imf', 'inro', 'irsg', 'isa', 'itc', 'iwc-whale', 'iwc-wheat', 'iwcc', 'iws', 'iwto', 'lafta', 'mfa', 'oapec', 'oecd', 'opec', 'un', 'unctad', 'who', 'worldbank', '']


268
['abdel-hadi-kandeel', 'alfonsin', 'alhaji-abdul-ahmed', 'alptemocin', 'amato', 'andersen', 'andriessen', 'aqazadeh', 'aquino', 'arafat', 'babangida', 'balladur', 'bangemann', 'barreto', 'b

In [120]:
from sklearn.datasets import fetch_20newsgroups,load_files
newsgroups_train = load_files('20news-bydate/20news-bydate-train/')

In [121]:
from pprint import pprint
pprint(list(newsgroups_train.target_names))
print newsgroups_train.filenames.shape
newsgroups_train.target

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']
(11314L,)


array([ 9,  4, 11, ..., 16, 18,  4])

In [122]:
for t in newsgroups_train.target[:10]:
    print(newsgroups_train.target_names[t])

rec.sport.baseball
comp.sys.mac.hardware
sci.crypt
comp.sys.mac.hardware
alt.atheism
comp.sys.mac.hardware
comp.windows.x
comp.windows.x
sci.med
sci.electronics


In [167]:
#Extracting features from text files
"""
If n_samples == 10000, storing X as a numpy array of type float32 would require 10000 x 100000 x 4 bytes = 4GB in RAM 
which is barely manageable on today’s computers.
Fortunately, most values in X will be zeros since for a given document less than a couple thousands of distinct words 
will be used. For this reason we say that bags of words are typically high-dimensional sparse datasets. We can save a lot
of memory by only storing the non-zero parts of the feature vectors in memory.
scipy.sparse matrices are data structures that do exactly this, and scikit-learn has built-in support for these structures.
"""
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
newsgroups_train.data = [s.decode("utf-8", errors='ignore') for s in newsgroups_train.data]
X_train_counts = count_vect.fit_transform(newsgroups_train.data)

In [168]:
X_train_counts.shape
count_vect.vocabulary_.get(u'algorithm')

27366

In [169]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130093)

In [170]:
#Applying multinomial naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf,newsgroups_train.target)

In [171]:
# lets say a new document comes
docs_new = ['God is love', ' mouth']
X_new_counts = count_vect.transform(docs_new)
X_new_counts.shape
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
X_new_tfidf
predicted = clf.predict(X_new_tfidf)

In [172]:
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, newsgroups_train.target_names[category]))

'God is love' => soc.religion.christian
' mouth' => sci.med


In [173]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
text_clf = text_clf.fit(newsgroups_train.data, newsgroups_train.target)

In [174]:
#Evaluation of the performance on the test set
newsgroups_test = load_files('20news-bydate/20news-bydate-test/')
newsgroups_test.data = [s.decode("utf-8", errors='ignore') for s in newsgroups_test.data]

In [179]:
predicted = text_clf.predict(newsgroups_test.data)
import numpy as np
np.mean(predicted==newsgroups_test.target)

0.7738980350504514

In [182]:
# learning a SVM classifier linear
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge',penalty = 'l2',alpha = 1e-3,n_iter=5,random_state=42))])
text_clf = text_clf.fit(newsgroups_train.data, newsgroups_train.target)

In [183]:
predicted = text_clf.predict(newsgroups_test.data)
import numpy as np
np.mean(predicted==newsgroups_test.target)

0.82461497610196499

In [188]:
#detailed performance analysis of the results:
from sklearn import metrics
print(metrics.classification_report(newsgroups_test.target, predicted,target_names=newsgroups_test.target_names))
print metrics.confusion_matrix(newsgroups_test.target, predicted)
"""
As expected the confusion matrix shows that posts from the newsgroups on atheism and christian are more often confused 
for one another than with computer graphics.
"""

                          precision    recall  f1-score   support

             alt.atheism       0.72      0.73      0.73       319
           comp.graphics       0.82      0.69      0.75       389
 comp.os.ms-windows.misc       0.71      0.78      0.75       394
comp.sys.ibm.pc.hardware       0.75      0.67      0.70       392
   comp.sys.mac.hardware       0.83      0.83      0.83       385
          comp.windows.x       0.84      0.76      0.80       395
            misc.forsale       0.81      0.91      0.86       390
               rec.autos       0.92      0.90      0.91       396
         rec.motorcycles       0.93      0.97      0.95       398
      rec.sport.baseball       0.89      0.90      0.90       397
        rec.sport.hockey       0.88      0.99      0.93       399
               sci.crypt       0.83      0.96      0.89       396
         sci.electronics       0.82      0.61      0.70       393
                 sci.med       0.89      0.86      0.87       396
         

'\nAs expected the confusion matrix shows that posts from the newsgroups on atheism and christian are more often confused \nfor one another than with computer graphics.\n'

In [189]:
#Parameter tuning using grid search
"""
Instead of tweaking the parameters of the various components of the chain, it is possible to run an exhaustive search of the
best parameters on a grid of possible values. We try out all classifiers on either words or bigrams, with or without idf, and 
with a penalty parameter of either 0.01 or 0.001 for the linear SVM:
"""
from sklearn.grid_search import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3),}
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [192]:
gs_clf = gs_clf.fit(newsgroups_train.data[:400], newsgroups_train.target[:400])

In [195]:
newsgroups_train.target_names[gs_clf.predict(['God is love'])]

C:\Users\neeru gupta\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  if __name__ == '__main__':


'alt.atheism'